In [1]:
import torch
torch.manual_seed(0)
import cvxpy as cp
import numpy as np
print(cp.installed_solvers())

['CVXOPT', 'ECOS', 'ECOS_BB', 'GLPK', 'GLPK_MI', 'OSQP', 'SCIPY', 'SCS']


In [13]:
# function to project a point q to the convex hull of D
def map_to_hull(D,q,x0,tol=1e-5):
  m,n = D.shape
  alpha = cp.Variable(m)
  objective = cp.Minimize(cp.sum_squares(alpha @ D - q))
  constraints = [0 <= alpha, cp.sum(alpha) == 1]
  prob = cp.Problem(objective, constraints)

  alpha.value = x0
  result = prob.solve(verbose=False, solver=cp.ECOS, warm_start=True, abstol=tol, feastol=tol, reltol=tol)

  alpha.value[alpha.value<0] = 0
  xp = torch.tensor(alpha.value).float() @ D
  dist = torch.linalg.norm(torch.tensor(q)-xp)
  return xp, dist, alpha.value

In [4]:
## Projection Operator
def projection(x,l,u):
  return np.maximum(np.minimum(x,u),l)

## Gradient Projection Operator
def gp(p,t1,t_bar):
  p[t_bar<=t1]=0
  return p

## tfinder Operator
def tfinder(t1,t2,p,c,D,x):
  f1 = c@p + (x@D) @ (np.transpose(D)@p)
  f2 = p@D @ (np.transpose(D) @ p)
  if f1>0:
    t = t1 # case 1
  elif -f1/f2>=0 and -f1/f2 < t2-t1:
    t = t1 - f1/f2 # case 2
  else:
    t = t2 # case 3
  return t

## KKT and residual
def kkt(x,D,c,l,u):
  temp=(D @ (np.transpose(D) @ x) + c)
  tolx = 1e-8
  # active = np.nonzero(x<tolx)
  # inactive = np.nonzero(x>tolx)
  sl = x-l # slack l
  yl = temp*1
  # yl[sl>tolx] = 0
  # yl[yl<0] = 0
  se = np.sum(x) - 1
  if np.abs(se)>tol:
    ye = 0
  else:
    ye = np.max(temp - yl)
  v = 0
  sl[sl==np.inf] = 1e5
  ##                           feasibility                                          complimentary                                      stationary                                             duality
  res = np.linalg.norm(np.minimum(sl,0))+np.linalg.norm(np.sum(x)-1)  +  np.linalg.norm(sl*yl)+np.linalg.norm(se*ye)  +  np.linalg.norm(D@(np.transpose(D)@x)+c-yl-ye)  +  np.linalg.norm(np.minimum(yl,0))+np.linalg.norm(np.minimum(ye,0))

  if res==0:
    v = 1 # KKT satistied

  return v, res, yl, ye

In [23]:
## QP function
def bqp_pg(c,D,l,u,x0,maxit,ssm,printlevel,tol,tolx,q):
  n = len(c)
  iter = 0
  x = projection(x0,l,u)
  stat = 1
  res = 10*tol
  obj = 0
  zl = []
  zu = zl

  # Functions
  def qf(z):
    return 0.5*z@D@np.transpose(D)@z + c@z
  def grad(z):
    return D@(np.transpose(D)@z) + c
  # g = @(z) (D*D'*z+c)./sum(D*D'*z+c)-1/n; % normalizing the gradient - not helpful
  if printlevel == 1:
    print("     iter  ,   obj  ,   residual ");

  while iter <= maxit:
    k,res,zl,zu = kkt(x,D,c,l,u)
    obj = qf(x)
    x_prev = x*1
    ## print
    if printlevel == 1:
      print(iter,"-",obj,"-",res)
    # KKT conditions
    if k==1:
      stat=0;
      print('KKT satisfied')
      break
    ## Residual tolerance
    if res<tol:
      stat=0
      print('residual satisfied',res)
      break
    ## Cauchy step
    t_bar = np.inf*(np.abs(x)+0.001)
    gg = grad(x)
    t1 = (x-u)/gg
    t2 = (x-l)/gg
    for i in range(n):
      if gg[i]<0 and u[i]<np.inf:
        t_bar[i] = t1[i]
      elif gg[i]>0 and u[i]>-np.inf:
        t_bar[i] = t2[i]
      else:
        t_bar[i+1] = np.inf
    t_sort = np.unique(t_bar[t_bar!=0])
    p = -1*gg
    t1 = 0
    xc = x
    for i in range(len(t_sort)):
      t2 = t_sort[i]
      p = gp(p,t1,t_bar)
      t = tfinder(t1,t2,p,c,D,xc)
      xc = xc + t*p
      if t>=t1 and t<t2:
          break
      t1=t2
    xc = projection(xc,l,u)
    ## Aproximate solution to accelerate
    if ssm==1:
      xc[xc<tolx] = 0
      xs = xc*1
      sl = xc-l #slack l
      active_set = np.where(sl == 0)[0]
      inactive_set = np.where(sl != 0)[0]
      if len(inactive_set)>0:
        Di = D[inactive_set,:]
        x0 = xc[inactive_set]
        _, _, xi = map_to_hull(Di,q,x0,tol*10)
        if printlevel == 1:
          print(len(xi))
        # print(len(x))
        xs[inactive_set] = xi*1
        xs[xs<tolx] = 0
        x = projection(xs,l,u)
      # print('%d inactive constraints out of %d total\n',sum(xs>0),n);
    else:
      x = xc*1
    ## change in solution
    if np.linalg.norm(x-x_prev) < tol and iter > 1:
      stat = 0
      print('no improvement in optimal solution')
      break
    ## maximum iteration
    iter+=1
    if iter==maxit+1:
      print('max iteration reached')

  return x,zl,zu,obj,res,iter,stat

In [43]:
# function to project a point q to the convex hull of D
def map_to_hull_pg(D,q,x0,tol=1e-5):
  n,d = D.shape
  c = -D@q
  l = np.zeros(n)
  u = np.ones(n)
  maxit = 100
  ssm = 1
  printlevel = 0
  tol = 1e-6
  tolx = 5e-4
  alpha,_,_,_,res,_,stat = bqp_pg(c,D,l,u,x0,maxit,ssm,printlevel,tol,tolx,q)
  xp = alpha @ D
  dist = np.linalg.norm(xp-q)

  return xp, dist, alpha

In [ ]:
%%time
import numpy as np
# fix the seed
np.random.seed(1)
maxit = 100
ssm = 1
printlevel = 0
tol = 1e-6
tolx = 5e-4
n = 50000
d = 64
D = np.random.randn(n,d)
q = np.random.randn(d)+1
c = -D@q
l = np.zeros(n)
u = np.ones(n)

In [ ]:
%%time
xopt, dist, alpha = map_to_hull(D,q,np.zeros(n),1e-5)
print("dist: ",dist)
print(np.min(alpha))
print(np.sum(alpha))

In [ ]:
%%time
xopt, dist, alpha = map_to_hull_pg(D,q,np.zeros(n),1e-5)
print("dist: ",dist)
print(np.min(alpha))
print(np.sum(alpha))

In [ ]:
%%time
x0 = np.zeros(n)
# x0(1:numel(xopt)) = xopt;
xopt,_,_,_,res,_,stat = bqp_pg(c,D,l,u,x0,maxit,ssm,printlevel,tol,tolx,q)
print(res)
print(stat)
dist = np.linalg.norm(xopt@D-q)
print("dist: ",dist)
print(np.min(xopt))
print(np.sum(xopt))

In [ ]:
%%time
# function to project a point q to the convex hull of D using the sketching method
def map_to_hull_sketch(D,q,nsketch):
  m,n = D.shape
  # alpha_torch = torch.zeros(0)
  alpha_np = np.zeros(0)
  mysize = 0
  maxit = 1
  ssm = 1
  printlevel = 0
  tol = 1e-5
  tolx = 5e-4
  for iter in range(nsketch):
    m_s = int(m/nsketch*(iter+1))
    alpha_np = np.pad(alpha_np, (0, m_s-mysize), 'constant')
    mysize = m_s*1
    Ds = D[0:mysize,:]
    # x0 = alpha_np*1
    l = np.zeros(mysize)
    u = np.ones(mysize)
    c = -Ds@q
    alpha_np,_,_,_,res,_,stat = bqp_pg(c,Ds,l,u,alpha_np,maxit,ssm,printlevel,tol,tolx,q)

  xp = alpha_np @ D
  dist = np.linalg.norm(q-xp)
  return xp, dist, alpha_np

In [ ]:
%%time
xp, dist, alpha = map_to_hull_sketch(D,q,3)
print(dist)
print(np.min(alpha))
print(np.sum(alpha))